In [1]:
### connect to colab-GPU machine
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Load package
from __future__ import print_function
from sklearn.model_selection import train_test_split
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import Model
import numpy as np

import cv2
import itertools
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, MaxPooling2D,BatchNormalization
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint

Using TensorFlow backend.


In [3]:
# Download competition data to colab

## if you have the data ignore this step
!wget -c https://dscomp.nkn.uidaho.edu/datadownload/Competition_Train_data_8000.npy
!wget -c https://dscomp.nkn.uidaho.edu/datadownload/Competition_Train_label_8000.npy

!wget -c https://dscomp.nkn.uidaho.edu/datadownload/Competition_Problems.npy

--2020-03-30 22:04:07--  https://dscomp.nkn.uidaho.edu/datadownload/Competition_Train_data_8000.npy
Resolving dscomp.nkn.uidaho.edu (dscomp.nkn.uidaho.edu)... 129.101.202.199
Connecting to dscomp.nkn.uidaho.edu (dscomp.nkn.uidaho.edu)|129.101.202.199|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

--2020-03-30 22:04:08--  https://dscomp.nkn.uidaho.edu/datadownload/Competition_Train_label_8000.npy
Resolving dscomp.nkn.uidaho.edu (dscomp.nkn.uidaho.edu)... 129.101.202.199
Connecting to dscomp.nkn.uidaho.edu (dscomp.nkn.uidaho.edu)|129.101.202.199|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

--2020-03-30 22:04:09--  https://dscomp.nkn.uidaho.edu/datadownload/Competition_Problems.npy
Resolving dscomp.nkn.uidaho.edu (dscomp.nkn.uidaho.edu)... 129.101.202.199
Connecting to dscomp.nk

In [4]:
# load data

### Train Data
Train_data = np.load('Competition_Train_data_8000.npy')  # Your directory
print(Train_data.shape)

### Train Label
Train_label = np.load('Competition_Train_label_8000.npy') 
print(Train_label.shape)

(8000, 64, 64)
(8000,)


In [5]:
# Prepare Training data

x_train, x_test, y_train, y_test = train_test_split(Train_data, Train_label, test_size=0.2, random_state=42)

x_train = np.reshape(x_train, [x_train.shape[0], x_train.shape[1], x_train.shape[2], 1])
x_test = np.reshape(x_test, [x_test.shape[0], x_test.shape[1], x_test.shape[2], 1])

x = np.reshape(Train_data, [Train_data.shape[0], Train_data.shape[1], Train_data.shape[2], 1])

print('Training set: {} and Training Targets: {}'.format(x_train.shape, y_train.shape))
print('Test set: {} and test targets: {}'.format(x_test.shape, y_test.shape))

## Generate one-hot-vector labels
y_train_onehot = keras.utils.to_categorical(y_train)
y_test_onehot = keras.utils.to_categorical(y_test)
y_onehot = keras.utils.to_categorical(Train_label)
# print('First 16 labels: ', y_train[:16])
# print('First 16 one-hot labels:\n', y_train_onehot[:16])

Training set: (6400, 64, 64, 1) and Training Targets: (6400,)
Test set: (1600, 64, 64, 1) and test targets: (1600,)


In [6]:
### New Colab with Tensorflow version 2.2 (Our Keras Codes works on 1.15.2)

# import tensorflow as tf
!pip install tensorflow-gpu==1.15.2

# Install Runable version (for our codes) tf package. 

In [7]:
import tensorflow as tf
tf.__version__

'1.15.2'

In [0]:
# A data augmenation package inside Keras

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(x_train)

In [9]:

model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (7,7),padding = 'Same', 
                activation ='relu', input_shape = (64,64,1)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(16, activation = "softmax"))

optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [0]:

# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=0, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [15]:

epochs = 10

history = model.fit_generator(datagen.flow(x_train,y_train_onehot, batch_size=64),
    epochs = epochs, steps_per_epoch = x.shape[0]//64,  
    validation_data = (x_test,y_test_onehot), callbacks=[learning_rate_reduction], verbose=1)

Epoch 1/10
125/125 [==============================] - 4s 34ms/step - loss: 0.6672 - acc: 0.7618 - val_loss: 0.4008 - val_acc: 0.8544
Epoch 2/10
125/125 [==============================] - 4s 34ms/step - loss: 0.6290 - acc: 0.7749 - val_loss: 0.4086 - val_acc: 0.8556
Epoch 3/10
125/125 [==============================] - 4s 34ms/step - loss: 0.5984 - acc: 0.7873 - val_loss: 0.3192 - val_acc: 0.8850
Epoch 4/10
125/125 [==============================] - 4s 33ms/step - loss: 0.5588 - acc: 0.7995 - val_loss: 0.3292 - val_acc: 0.8775
Epoch 5/10
125/125 [==============================] - 4s 34ms/step - loss: 0.5034 - acc: 0.8169 - val_loss: 0.3012 - val_acc: 0.8788
Epoch 6/10
125/125 [==============================] - 4s 34ms/step - loss: 0.4769 - acc: 0.8340 - val_loss: 0.2595 - val_acc: 0.9012
Epoch 7/10
125/125 [==============================] - 4s 34ms/step - loss: 0.4596 - acc: 0.8381 - val_loss: 0.2532 - val_acc: 0.9081
Epoch 8/10
125/125 [==============================] - 4s 34ms/step - 

In [0]:
# from keras.models import load_model
# model = load_model('bestmodel_v6.h5')

In [0]:
# test data
Test_data = np.load('Competition_Problems.npy') 
Test_data.shape

In [19]:
rec = []
Testsize = 20000
for i in range(Testsize):
    if i % 1000 == 0:
        print(i)
    img = Test_data[i][:,250:322]
    img = cv2.resize(img, (64,64))
    img_ = np.reshape(img, [1, img.shape[0], img.shape[1], 1])

    y_test_pred = model.predict(img_, verbose=0)
    y_test_pred = np.argmax(y_test_pred,axis = 1)   

    img2 = Test_data[i][:,315:384]
    img2 = cv2.resize(img2, (64,64))    
    img2_ = np.reshape(img2, [1, img2.shape[0], img2.shape[1], 1])    

    y_test_pred2 = model.predict(img2_, verbose=0)
    y_test_pred2 = np.argmax(y_test_pred2,axis = 1)    
    
    img3 = Test_data[i][:,60:130]
    img3 = cv2.resize(img3, (64,64))    
    img3_ = np.reshape(img3, [1, img3.shape[0], img3.shape[1], 1])    

    y_test_pred3 = model.predict(img3_, verbose=0)
    y_test_pred3 = np.argmax(y_test_pred3,axis = 1)    

    # grading
    ## equ 1
    if y_test_pred3 == 0:
        if y_test_pred == 7:
            if y_test_pred2 == 11:
                rec.append(10)
            else:
                rec.append(9) 
                
        elif y_test_pred == 12:
            if y_test_pred2 == 11:
                rec.append(5)
            else:
                rec.append(0)
                
        else:
            rec.append(0)
            
    ### equ 2   
    elif y_test_pred3 == 1:
        if y_test_pred == 8:
            if y_test_pred2 == 11:
                rec.append(10)
            else:
                rec.append(9) 
                
        elif y_test_pred == 13:
            if y_test_pred2 == 11:
                rec.append(5)
            else:
                rec.append(0)

        elif y_test_pred == 4:
            if y_test_pred2 == 11:
                rec.append(2)
            else:
                rec.append(1)
        else:
            rec.append(0) 
    
    ### equ 3 
    elif y_test_pred3 == 2:
        if y_test_pred == 9:
            if y_test_pred2 == 11:
                rec.append(10)
            else:
                rec.append(9) 
                
        elif y_test_pred == 14:
            if y_test_pred2 == 11:
                rec.append(5)
            else:
                rec.append(0)

        elif y_test_pred == 5:
            if y_test_pred2 == 11:
                rec.append(2)
            else:
                rec.append(1)
        else:
            rec.append(0) 

    ### equ 4 
    elif y_test_pred3 == 3:
        if y_test_pred == 10:
            if y_test_pred2 == 11:
                rec.append(10)
            else:
                rec.append(9) 
                
        elif y_test_pred == 15:
            if y_test_pred2 == 11:
                rec.append(5)
            else:
                rec.append(0)

        elif y_test_pred == 6:
            if y_test_pred2 == 11:
                rec.append(2)
            else:
                rec.append(1)
        else:
            rec.append(0) 

    ### equ 5 
    elif y_test_pred3 == 4:
        if y_test_pred == 1:
            if y_test_pred2 == 11:
                rec.append(10)
            else:
                rec.append(9) 
        else:
            rec.append(0) 
            
    ### equ 6 
    elif y_test_pred3 == 5:
        if y_test_pred == 2:
            if y_test_pred2 == 11:
                rec.append(10)
            else:
                rec.append(9) 
        else:
            rec.append(0) 
            
    ### equ 7 
    elif y_test_pred3 == 6:
        if y_test_pred == 3:
            if y_test_pred2 == 11:
                rec.append(10)
            else:
                rec.append(9) 
        else:
            rec.append(0) 
    
    else:
        rec.append(0)
            
print(len(rec))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000


In [2]:
# Transfer to numpy
result = np.array(rec)

## save to your own directory
np.save('/content/drive/My Drive/2020datacamp/rec_24_cnn.npy',result)

NameError: ignored

In [0]:
# Now your are ready to submit 
